In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import os
import datetime
from sklearn.model_selection import train_test_split

In [3]:
def extend_vector_with_weekday(hubs,cos,sin):
    l = []
    for hub in range(len(hubs)):
        h = list(hubs[hub])
        h.extend([sin[hub],cos[hub]])
        l.append(np.array(h))
    return l

In [4]:
data = pd.read_pickle('DR_Data/pickup_series_10max_zones.pickle')
pickups_out = np.array(data.Hubs)
weekday = True
added_features = 0

if weekday:
    data.Hubs = extend_vector_with_weekday(data.Hubs,data.weekday_sin,data.weekday_cos)
    added_features = added_features + 2
    
pickups_in = np.array(data.Hubs)

num_zones = pickups_in[0].shape[0] - added_features # -2 from weekdays sin and cos
seq_length = 28
num_intervals = pickups_in.shape[0]-seq_length
num_features = pickups_in[0].shape[0]
validation_size = 0.1

X = np.zeros((num_features, seq_length, num_intervals))
y = np.zeros((num_zones, 1, num_intervals))
for i in range(num_intervals):
    X[:,:,i] = np.array(list(pickups_in[i:i+seq_length])).T
    y[:,:,i] = np.array([pickups_out[i+seq_length]]).T
    
print(X.shape,y.shape)
X_train, X_valid, y_train, y_valid = train_test_split(X.T, y.T, test_size=validation_size, random_state=1)
print(X_train.shape, y_train.shape)

(12, 28, 1551) (10, 1, 1551)
(1395, 28, 12) (1395, 1, 10)


In [5]:
# Load functions
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, GRU, Conv2d, Dropout, Dropout2d, MaxPool2d, BatchNorm1d, BatchNorm2d
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax

In [6]:
use_cuda = torch.cuda.is_available()
print("Running GPU.") if use_cuda else print("No GPU available.")
#use_cuda =False

def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if use_cuda:
        return x.cuda()
    return x


def get_numpy(x):
    """ Get numpy array for both cuda and not. """
    if use_cuda:
        return x.cpu().data.numpy()
    return x.data.numpy()

Running GPU.


In [21]:
rnn_seq_length = seq_length
rnn_input_size = num_features
rnn_hidden_size = 16
rnn_direction = 0
rnn_layers = 2

rnn_out_features = rnn_seq_length*rnn_hidden_size*(rnn_direction+1)
features_cat_size = rnn_out_features

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()        
        #self.drop = Dropout(p=0.5)
        self.bn0 = BatchNorm1d(rnn_seq_length)
        
        self.rnn_1 = nn.LSTM(input_size=rnn_input_size,
                            hidden_size=rnn_hidden_size,
                            num_layers=rnn_layers,
                            bidirectional=(rnn_direction>0),
                            dropout=0.5)
        
        self.l_out = Linear(in_features=features_cat_size,
                            out_features=num_zones,
                            bias=True)
        
    def forward(self, x):
        features = []
        out = {}
        #x = self.bn0(x)
        
        features_rnn = x.view(rnn_seq_length, -1, rnn_input_size)
        features_rnn,_ = self.rnn_1(features_rnn)
        #features_rnn = self.drop(features_rnn)
        
        features_rnn = features_rnn.view(-1, rnn_out_features) #self.rnn_1.hidden_size)
        
        # Append features to the list "features"
        features.append(features_rnn)
        
        ## Output layer where all features are in use ##
        features_final = torch.cat(features, dim=1)
        
        #features_final = self.drop(features_final)
        out['out'] = self.l_out(features_final)
        '''
        x, (h,c) = self.rnn_1(x)
        #x = x.view(-1,self.rnn_1.rnn_hidden_size)
        x = self.l_out(x)
        return x
        '''
        return out['out']

net = Net()
if use_cuda:
    net.cuda()
print(net)

Net(
  (bn0): BatchNorm1d(28, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (rnn_1): LSTM(12, 16, num_layers=2, dropout=0.5)
  (l_out): Linear(in_features=448, out_features=10, bias=True)
)


In [ ]:
# Hyper-parameters
print(X_valid.shape)
# setting hyperparameters and gettings epoch sizes
batch_size = 32
num_epochs = 5000
num_samples_train = X_train.shape[0]
num_batches_train = num_samples_train // batch_size
num_samples_valid = X_valid.shape[0]
num_batches_valid = num_samples_valid // batch_size

# Define a loss function and optimizer for this problem
criterion = torch.nn.MSELoss(size_average=None, reduce=None, reduction='mean')
optimizer = optim.SGD(net.parameters(), lr=0.0005,momentum=0.2, weight_decay=3e-3)

def accuracy(ys, ts):
    predictions = torch.round(ys)
    correct_prediction = torch.eq(predictions,ts)
    return torch.mean(correct_prediction.float())

# Track loss
train_iter = []
train_loss, train_accs = [], []
valid_iter = []
valid_loss, valid_accs = [], []

# For each epoch
for i in range(num_epochs):
    
    # Track loss
    epoch_training_loss = 0
    epoch_training_accs = 0
    epoch_validation_loss = 0
    epoch_validation_accs = 0
    
    net.eval()
    for j in range(num_batches_valid):
        inputs = get_variable(torch.Tensor(X_valid[j * batch_size: (j + 1) * batch_size]))
        targets = get_variable(torch.Tensor(np.squeeze(y_valid[j * batch_size: (j + 1) * batch_size])))
        # Forward pass
        outputs = net.forward(inputs)
        # Compute loss
        loss = criterion(outputs,targets)
        # Update loss
        epoch_validation_loss += loss.cpu().detach().numpy()
        epoch_validation_accs += accuracy(outputs, targets)
    
    net.train()
    for j in range(num_batches_train):
        inputs = get_variable(torch.Tensor(X_train[j * batch_size: (j + 1) * batch_size]))
        targets = get_variable(torch.Tensor(np.squeeze(y_train[j * batch_size: (j + 1) * batch_size])))
        outputs = net.forward(inputs)
        # Compute loss
        loss = criterion(outputs,targets)
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Update loss
        epoch_training_loss += loss.cpu().detach().numpy()
        epoch_training_accs += accuracy(outputs,targets)
        
    # Save loss for plot
    train_loss.append(epoch_training_loss/num_batches_train)
    train_accs.append(epoch_training_accs/num_batches_train)
    valid_loss.append(epoch_validation_loss/num_batches_valid)
    valid_accs.append(epoch_validation_accs/num_batches_valid)

    # Print loss every 5 epochs
    if i % 5 == 0:
        print(f'Epoch {i} \t train loss: {train_loss[-1]:.10f} \t valid loss: {valid_loss[-1]:.10f} \t valid acc: {valid_accs[-1]:.4f}')



# Plot training and validation loss
epoch = np.arange(len(train_loss))
plt.figure()
plt.subplot(1,2,1)
plt.plot(epoch, train_loss, 'r', label='Training loss',)
plt.plot(epoch, valid_loss, 'b', label='Validation loss')
plt.legend()
plt.xlabel('Epoch'), plt.ylabel('MSE')

plt.subplot(1,2,2)
plt.plot(epoch, train_accs, 'r', label='Training Accuracy')
plt.plot(epoch, valid_accs, 'b', label='Validation Accuracy')
plt.legend()
plt.xlabel('Epoch'), plt.ylabel('')
plt.show()

(156, 28, 12)
Epoch 0 	 train loss: 12.3434626890 	 valid loss: 13.0148048401 	 valid acc: 0.4227
Epoch 5 	 train loss: 10.9627798657 	 valid loss: 11.5877821445 	 valid acc: 0.4242
Epoch 10 	 train loss: 9.2213425803 	 valid loss: 9.6816856861 	 valid acc: 0.1977
Epoch 15 	 train loss: 8.5741094046 	 valid loss: 8.8551368713 	 valid acc: 0.1469
Epoch 20 	 train loss: 8.4999211555 	 valid loss: 8.7296581268 	 valid acc: 0.1336
Epoch 25 	 train loss: 8.4771340947 	 valid loss: 8.7082539797 	 valid acc: 0.1297
Epoch 30 	 train loss: 8.4727326437 	 valid loss: 8.6954855919 	 valid acc: 0.1289
Epoch 35 	 train loss: 8.4556183205 	 valid loss: 8.6826483011 	 valid acc: 0.1266
Epoch 40 	 train loss: 8.4480491904 	 valid loss: 8.6690151691 	 valid acc: 0.1250
Epoch 45 	 train loss: 8.4443492390 	 valid loss: 8.6543456316 	 valid acc: 0.1250
Epoch 50 	 train loss: 8.4386819795 	 valid loss: 8.6383278370 	 valid acc: 0.1258
Epoch 55 	 train loss: 8.4091983008 	 valid loss: 8.6207451820 	 valid 

Epoch 490 	 train loss: 4.8324009041 	 valid loss: 4.5776026249 	 valid acc: 0.3336
Epoch 495 	 train loss: 4.8120252698 	 valid loss: 4.5715532303 	 valid acc: 0.3344
Epoch 500 	 train loss: 4.8482732218 	 valid loss: 4.5627725720 	 valid acc: 0.3344
Epoch 505 	 train loss: 4.8047176350 	 valid loss: 4.5551797748 	 valid acc: 0.3359
Epoch 510 	 train loss: 4.8391234431 	 valid loss: 4.5487131476 	 valid acc: 0.3359
Epoch 515 	 train loss: 4.8412802219 	 valid loss: 4.5445135832 	 valid acc: 0.3328
Epoch 520 	 train loss: 4.8474614620 	 valid loss: 4.5375386477 	 valid acc: 0.3367
Epoch 525 	 train loss: 4.7855143270 	 valid loss: 4.5313993096 	 valid acc: 0.3359
Epoch 530 	 train loss: 4.7660782282 	 valid loss: 4.5267505646 	 valid acc: 0.3375
Epoch 535 	 train loss: 4.7686104719 	 valid loss: 4.5219354033 	 valid acc: 0.3406
Epoch 540 	 train loss: 4.7903076992 	 valid loss: 4.5182954669 	 valid acc: 0.3375
Epoch 545 	 train loss: 4.7922909814 	 valid loss: 4.5223445296 	 valid acc:

Epoch 980 	 train loss: 4.5298882695 	 valid loss: 4.3409779072 	 valid acc: 0.3453
Epoch 985 	 train loss: 4.5251233356 	 valid loss: 4.3412507176 	 valid acc: 0.3477
Epoch 990 	 train loss: 4.5592420268 	 valid loss: 4.3453749418 	 valid acc: 0.3453
Epoch 995 	 train loss: 4.5304465904 	 valid loss: 4.3472608924 	 valid acc: 0.3477
Epoch 1000 	 train loss: 4.5557097612 	 valid loss: 4.3415541649 	 valid acc: 0.3477
Epoch 1005 	 train loss: 4.5166178748 	 valid loss: 4.3389524221 	 valid acc: 0.3406
Epoch 1010 	 train loss: 4.5287989406 	 valid loss: 4.3365822434 	 valid acc: 0.3461
Epoch 1015 	 train loss: 4.4981802785 	 valid loss: 4.3335049748 	 valid acc: 0.3477
Epoch 1020 	 train loss: 4.5442878867 	 valid loss: 4.3347629309 	 valid acc: 0.3445
Epoch 1025 	 train loss: 4.5550735496 	 valid loss: 4.3330408931 	 valid acc: 0.3477
Epoch 1030 	 train loss: 4.5435920926 	 valid loss: 4.3314006329 	 valid acc: 0.3438
Epoch 1035 	 train loss: 4.4830773930 	 valid loss: 4.3309965730 	 va

Epoch 1465 	 train loss: 4.4874575582 	 valid loss: 4.3005818129 	 valid acc: 0.3531
Epoch 1470 	 train loss: 4.4169881677 	 valid loss: 4.2960969806 	 valid acc: 0.3484
Epoch 1475 	 train loss: 4.4159444487 	 valid loss: 4.2973921299 	 valid acc: 0.3516
Epoch 1480 	 train loss: 4.4342732984 	 valid loss: 4.2963302135 	 valid acc: 0.3508
Epoch 1485 	 train loss: 4.4077809023 	 valid loss: 4.2964328527 	 valid acc: 0.3477
Epoch 1490 	 train loss: 4.4742209800 	 valid loss: 4.2941518426 	 valid acc: 0.3508
Epoch 1495 	 train loss: 4.4318601863 	 valid loss: 4.2919441462 	 valid acc: 0.3484
Epoch 1500 	 train loss: 4.4188400978 	 valid loss: 4.3000716567 	 valid acc: 0.3492
Epoch 1505 	 train loss: 4.4484747676 	 valid loss: 4.2961360216 	 valid acc: 0.3531
Epoch 1510 	 train loss: 4.4268323876 	 valid loss: 4.2952976227 	 valid acc: 0.3461
Epoch 1515 	 train loss: 4.4351542717 	 valid loss: 4.2918362617 	 valid acc: 0.3539
Epoch 1520 	 train loss: 4.3980761018 	 valid loss: 4.2875115275 

Epoch 1950 	 train loss: 4.3719546074 	 valid loss: 4.2691670060 	 valid acc: 0.3453
Epoch 1955 	 train loss: 4.3698636709 	 valid loss: 4.2651998997 	 valid acc: 0.3484
Epoch 1960 	 train loss: 4.3294925856 	 valid loss: 4.2659943700 	 valid acc: 0.3438
Epoch 1965 	 train loss: 4.3962045270 	 valid loss: 4.2636640072 	 valid acc: 0.3445
Epoch 1970 	 train loss: 4.3913139133 	 valid loss: 4.2601923347 	 valid acc: 0.3500
Epoch 1975 	 train loss: 4.3457405290 	 valid loss: 4.2631229162 	 valid acc: 0.3461
Epoch 1980 	 train loss: 4.3632475221 	 valid loss: 4.2729855776 	 valid acc: 0.3461
Epoch 1985 	 train loss: 4.3494820207 	 valid loss: 4.2604335546 	 valid acc: 0.3461
Epoch 1990 	 train loss: 4.3652750503 	 valid loss: 4.2632695436 	 valid acc: 0.3453
Epoch 1995 	 train loss: 4.3183308202 	 valid loss: 4.2679557204 	 valid acc: 0.3477
Epoch 2000 	 train loss: 4.3421893896 	 valid loss: 4.2720269561 	 valid acc: 0.3492
Epoch 2005 	 train loss: 4.3145859574 	 valid loss: 4.2642872930 

Epoch 2435 	 train loss: 4.2860363805 	 valid loss: 4.2439427972 	 valid acc: 0.3469
Epoch 2440 	 train loss: 4.3098401580 	 valid loss: 4.2567154765 	 valid acc: 0.3430
Epoch 2445 	 train loss: 4.3109145664 	 valid loss: 4.2497533560 	 valid acc: 0.3445
Epoch 2450 	 train loss: 4.2856134038 	 valid loss: 4.2472531199 	 valid acc: 0.3430
Epoch 2455 	 train loss: 4.2286542626 	 valid loss: 4.2502418160 	 valid acc: 0.3430
Epoch 2460 	 train loss: 4.3073774548 	 valid loss: 4.2646985054 	 valid acc: 0.3383
Epoch 2465 	 train loss: 4.2787637433 	 valid loss: 4.2486873865 	 valid acc: 0.3430
Epoch 2470 	 train loss: 4.2592806040 	 valid loss: 4.2432086468 	 valid acc: 0.3422
Epoch 2475 	 train loss: 4.2534875870 	 valid loss: 4.2406216860 	 valid acc: 0.3445
Epoch 2480 	 train loss: 4.2922847825 	 valid loss: 4.2416529655 	 valid acc: 0.3461
Epoch 2485 	 train loss: 4.2635853457 	 valid loss: 4.2375959158 	 valid acc: 0.3469
Epoch 2490 	 train loss: 4.2622098923 	 valid loss: 4.2453745008 

Epoch 2920 	 train loss: 4.1887434638 	 valid loss: 4.2155413032 	 valid acc: 0.3430
Epoch 2925 	 train loss: 4.2044234664 	 valid loss: 4.2205806971 	 valid acc: 0.3453
Epoch 2930 	 train loss: 4.1625230700 	 valid loss: 4.2124108076 	 valid acc: 0.3477
Epoch 2935 	 train loss: 4.2020769785 	 valid loss: 4.2198703289 	 valid acc: 0.3484
Epoch 2940 	 train loss: 4.1928583411 	 valid loss: 4.2252908945 	 valid acc: 0.3461
Epoch 2945 	 train loss: 4.1725039316 	 valid loss: 4.2363021970 	 valid acc: 0.3453
Epoch 2950 	 train loss: 4.1981837029 	 valid loss: 4.2118260264 	 valid acc: 0.3445
Epoch 2955 	 train loss: 4.2163317758 	 valid loss: 4.2204607129 	 valid acc: 0.3461
Epoch 2960 	 train loss: 4.1675267109 	 valid loss: 4.2262032628 	 valid acc: 0.3453
Epoch 2965 	 train loss: 4.2118188670 	 valid loss: 4.2136043906 	 valid acc: 0.3469
Epoch 2970 	 train loss: 4.1769208742 	 valid loss: 4.2176504135 	 valid acc: 0.3445
Epoch 2975 	 train loss: 4.1961786414 	 valid loss: 4.2157086134 

Epoch 3405 	 train loss: 4.0809549010 	 valid loss: 4.2219447494 	 valid acc: 0.3313
Epoch 3410 	 train loss: 4.1086711662 	 valid loss: 4.2113816738 	 valid acc: 0.3344
Epoch 3415 	 train loss: 4.1664775416 	 valid loss: 4.2005615234 	 valid acc: 0.3375
Epoch 3420 	 train loss: 4.1166983316 	 valid loss: 4.2160872221 	 valid acc: 0.3352
Epoch 3425 	 train loss: 4.0653869607 	 valid loss: 4.2189937234 	 valid acc: 0.3375
Epoch 3430 	 train loss: 4.1194869418 	 valid loss: 4.2038389444 	 valid acc: 0.3359
Epoch 3435 	 train loss: 4.1097669990 	 valid loss: 4.1950494647 	 valid acc: 0.3367
Epoch 3440 	 train loss: 4.0850865120 	 valid loss: 4.1982666850 	 valid acc: 0.3352
Epoch 3445 	 train loss: 4.0994258260 	 valid loss: 4.2104794383 	 valid acc: 0.3352
Epoch 3450 	 train loss: 4.0952711660 	 valid loss: 4.2067479491 	 valid acc: 0.3336
Epoch 3455 	 train loss: 4.1314848190 	 valid loss: 4.2088178992 	 valid acc: 0.3328
Epoch 3460 	 train loss: 4.1016249490 	 valid loss: 4.2070726752 

In [20]:
print(targets)
print(outputs)
print(torch.round(outputs))
print(torch.div(targets-torch.round(outputs),targets+0.001))

tensor([[ 7.,  8.,  5.,  4., 10.,  1.,  6.,  6.,  5.,  3.],
        [ 0.,  2.,  1.,  2.,  6.,  2.,  0.,  6.,  1.,  0.],
        [ 1.,  3.,  6.,  2.,  4.,  3.,  2.,  2.,  2.,  1.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.],
        [ 3.,  4.,  9.,  8.,  4.,  0.,  0.,  3.,  0.,  4.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [15.,  9., 11., 14., 14., 10.,  8., 11.,  6.,  8.],
        [ 4.,  0.,  0.,  0.,  0.,  3.,  2.,  3.,  0.,  1.],
        [ 0.,  3.,  1.,  1.,  0.,  3.,  1.,  0.,  0.,  0.],
        [ 1.,  1.,  0.,  0.,  3.,  0.,  0.,  0.,  1.,  0.],
        [ 1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [10.,  2.,  0.,  2.,  2.,  2.,  0.,  1.,  0.,  2.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 9.,  2.,  3.,  1., 10.,  1.,  2.,  8.,  3.,  7.],
        [ 4.,  5.,  1.,  1.,  3.,  1.,  2.,  2.,  3.,  2.],
        [ 1.,  3., 11.,  4.,  4.,  2.,  2.,  2.,  4.,  7.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  